In [1]:
# training a model using the validation_split in model arguments

In [2]:
# reading the data ml-latest-small
import pandas as pd
import zipfile
import numpy as np 
zf = zipfile.ZipFile('/home/elena/Downloads/ml-latest-small.zip')
# reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(zf.open('ml-latest-small/ratings.csv'), names=r_cols)
m_cols=['movie_id', 'title', 'genre']
movies = pd.read_csv(zf.open('ml-latest-small/movies.csv'), names=m_cols)
# merging ratings and movies
data=pd.merge(ratings,movies,on='movie_id')
data.head()

,user_id,movie_id,rating,unix_timestamp,title,genre
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [3]:
zz = zipfile.ZipFile('/home/elena/Downloads/ml-100k.zip')
# reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(zz.open('ml-100k/u.user'), sep='|', names=u_cols,encoding='latin-1')
data=pd.merge(users,data, on='user_id')
data.head()

,user_id,age,sex,occupation,zip_code,movie_id,rating,unix_timestamp,title,genre
0,1,24,M,technician,85711,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,24,M,technician,85711,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,24,M,technician,85711,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,24,M,technician,85711,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,24,M,technician,85711,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [4]:
data.movie_id.unique().shape[0], data.user_id.unique().shape[0], data.shape

(9724, 610, (100836, 10))

In [5]:
# Implementation 
from keras.layers import Input, Embedding, Concatenate, Flatten, Dense, Dot, Add, Multiply, Subtract, Average
from keras.models import Model
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [6]:
from tensorflow import random 
np.random.seed(42)
random.set_seed(42)

In [7]:
def embedding_model(hidden_units, user_embedding_dim, user_max_cat_value, movie_embedding_dim, movie_max_cat_value,merging_method):
    # Each instance will consist of two inputs: a single user id, and a single movie id
    user_id_input = Input(shape=(1,), name='user_id')
    movie_id_input = Input(shape=(1,), name='movie_id')
    # Embeddings
    user_embedded = Embedding(user_max_cat_value+1, user_embedding_dim, 
                                       input_length=1, name='user_embedding')(user_id_input)
    movie_embedded = Embedding(movie_max_cat_value+1, movie_embedding_dim, 
                                        input_length=1, name='movie_embedding')(movie_id_input)
    # merging the embeddings
    if merging_method=='concatenate':
        merged = Concatenate()([user_embedded, movie_embedded])
    if merging_method=='dot_product':
        merged =Dot(name = 'dot_product', normalize = True, axes = 2)([user_embedded, movie_embedded])
    if merging_method=='add':
        merged =Add()([user_embedded, movie_embedded])
    if merging_method=='substract':
        merged=Subtract()([user_embedded, movie_embedded])
    if merging_method=='multiply':
        merged=Multiply()([user_embedded, movie_embedded])
    if merging_method=='average':
        merged=Average()([user_embedded, movie_embedded])
    out = Flatten()(merged)

    # Add one or more hidden layers
    for n_hidden in hidden_units:
        out = Dense(n_hidden, activation='relu')(out)

    # A single output: our predicted rating
    out = Dense(1, activation='linear', name='prediction')(out)
    return Model(inputs = [user_id_input, movie_id_input],outputs = out)

In [8]:
hidden_units = (100,50) #same as in pytorch model
movie_embedding_dim = 50 #same as in pytorch model
user_embedding_dim = 50  #same as in pytorch model
user_max_cat_value = data.user_id.max()
movie_max_cat_value=data.movie_id.max()
model_concatenate=embedding_model(hidden_units, user_embedding_dim, user_max_cat_value, movie_embedding_dim, movie_max_cat_value, merging_method='concatenate')
# model_concatenate.summary(line_length=88)

In [9]:
# compiling 
model_concatenate.compile(optimizer = 'Adam',loss='MSE',metrics=['MAE'])
# early stopping
es=EarlyStopping(monitor='val_MAE', min_delta=0, patience=0, verbose=0, mode='min', baseline=None, restore_best_weights=False)
# training and using the pre-defined train and test data
trained_model_concatenate= model_concatenate.fit(x=[data.user_id, data.movie_id], y=data.rating, batch_size=500,epochs=10, verbose=2, validation_split=0.25, callbacks=[es])

Train on 75627 samples, validate on 25209 samples
Epoch 1/10
 - 99s - loss: 3.6265 - MAE: 1.4042 - val_loss: 3.4668 - val_MAE: 1.6567
Epoch 2/10
 - 94s - loss: 0.7252 - MAE: 0.6575 - val_loss: 3.4317 - val_MAE: 1.6487
Epoch 3/10
 - 96s - loss: 0.6879 - MAE: 0.6374 - val_loss: 3.3224 - val_MAE: 1.6182
Epoch 4/10
 - 97s - loss: 0.6722 - MAE: 0.6288 - val_loss: 3.1830 - val_MAE: 1.5798
Epoch 5/10
 - 96s - loss: 0.6622 - MAE: 0.6230 - val_loss: 3.0031 - val_MAE: 1.5282
Epoch 6/10
 - 95s - loss: 0.6513 - MAE: 0.6163 - val_loss: 2.7501 - val_MAE: 1.4518
Epoch 7/10
 - 93s - loss: 0.6337 - MAE: 0.6063 - val_loss: 2.5357 - val_MAE: 1.3839
Epoch 8/10
 - 94s - loss: 0.6150 - MAE: 0.5966 - val_loss: 2.3147 - val_MAE: 1.3101
Epoch 9/10
 - 92s - loss: 0.5966 - MAE: 0.5863 - val_loss: 2.1117 - val_MAE: 1.2442
Epoch 10/10
 - 93s - loss: 0.5781 - MAE: 0.5753 - val_loss: 2.0007 - val_MAE: 1.2024


In [10]:
model_dot=embedding_model(hidden_units, user_embedding_dim, user_max_cat_value, movie_embedding_dim, movie_max_cat_value, merging_method='dot_product')

In [11]:
# compiling 
model_dot.compile(optimizer = 'Adam',loss='MSE',metrics=['MAE'])
# early stopping
es=EarlyStopping(monitor='val_MAE', min_delta=0, patience=0, verbose=0, mode='min', baseline=None, restore_best_weights=False)
# training and using the pre-defined train and test data
trained_model_dot= model_dot.fit(x=[data.user_id, data.movie_id], y=data.rating, batch_size=500,epochs=10, verbose=2, validation_split=0.25, callbacks=[es])

Train on 75627 samples, validate on 25209 samples
Epoch 1/10
 - 107s - loss: 4.2082 - MAE: 1.5940 - val_loss: 1.1414 - val_MAE: 0.8415
Epoch 2/10
 - 104s - loss: 1.0033 - MAE: 0.7955 - val_loss: 1.2040 - val_MAE: 0.8767


In [12]:
model_add=embedding_model(hidden_units, user_embedding_dim, user_max_cat_value, movie_embedding_dim, movie_max_cat_value, merging_method='add')
# compiling 
model_add.compile(optimizer = 'Adam',loss='MSE',metrics=['MAE'])
# early stopping
es=EarlyStopping(monitor='val_MAE', min_delta=0, patience=0, verbose=0, mode='min', baseline=None, restore_best_weights=False)
# training and using the pre-defined train and test data
trained_model_add= model_add.fit(x=[data.user_id, data.movie_id], y=data.rating, batch_size=500,epochs=10, verbose=2, validation_split=0.25, callbacks=[es])

Train on 75627 samples, validate on 25209 samples
Epoch 1/10
 - 76s - loss: 3.1967 - MAE: 1.3082 - val_loss: 2.7149 - val_MAE: 1.4434
Epoch 2/10
 - 82s - loss: 0.7240 - MAE: 0.6571 - val_loss: 2.5217 - val_MAE: 1.3842
Epoch 3/10
 - 87s - loss: 0.6853 - MAE: 0.6362 - val_loss: 2.4013 - val_MAE: 1.3467
Epoch 4/10
 - 83s - loss: 0.6690 - MAE: 0.6267 - val_loss: 2.2061 - val_MAE: 1.2820
Epoch 5/10
 - 75s - loss: 0.6548 - MAE: 0.6190 - val_loss: 2.0157 - val_MAE: 1.2175
Epoch 6/10
 - 74s - loss: 0.6422 - MAE: 0.6112 - val_loss: 1.9233 - val_MAE: 1.1855
Epoch 7/10
 - 70s - loss: 0.6208 - MAE: 0.5996 - val_loss: 1.8407 - val_MAE: 1.1544
Epoch 8/10
 - 94s - loss: 0.5866 - MAE: 0.5802 - val_loss: 1.8014 - val_MAE: 1.1367
Epoch 9/10
 - 94s - loss: 0.5475 - MAE: 0.5581 - val_loss: 1.8783 - val_MAE: 1.1625
